# 03 Thai Only Augmentations

- Input: `02_backtranslate_english.csv`
- Output `03_thai_augmentations.csv`

In [1]:
import gc

import nltk
import pandas as pd

from tqdm.notebook import tqdm
from pythainlp.augment import WordNetAug, word2vec, lm
nltk.download('omw-1.4');

[nltk_data] Downloading package omw to /Users/parinzee/nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/parinzee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/parinzee/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Load Dataset from Step 1

In [2]:
dataset = pd.read_csv('data/01_prepare_dataset.csv')
dataset

,question,context,answers,source,id
0,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,82c589fda5d3007be968f303e04a9ac58953299a64a5be...
1,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...
2,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...
3,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...
4,บิดาของคลีโอพัตราเป็นใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...
...,...,...,...,...,...
16975,หนังสือการ์ตูนชานะ นักรบเนตรอัคคี มีกี่เล่ม?,นิยาย ชานะ นักรบเนตรอัคคี แต่งโดย ยาชิจิโร ทาค...,"{'text': ['22', '26'], 'answer_start': [267, 9...",tydiqa,e780cc4812dd88f21ec7f0152905b15588d1a6faab66d2...
16976,ไบโอช็อก อินฟินิต เปิดตัวครั้งแรกเมื่อไหร่?,ไบโอช็อก อินฟินิต (English: BioShock Infinite)...,"{'text': ['26 มีนาคม พ.ศ. 2556', '26 มีนาคม พ....",tydiqa,dd424979403f3909773a832c4d83818e564fe7bfeb4f38...
16977,ยู ซึง-โฮ เริ่มเข้าวงการบันเทิงเมื่อไหร่?,ยู ซึง-โฮเดบิวต์เมื่อปี 1999 กับผลงานโฆษณาของ ...,"{'text': ['ปี 1999', '1999'], 'answer_start': ...",tydiqa,d131cf44b5a08630e822cbf4c33576a434095f6ff4184f...
16978,นภัทร อินทร์ใจเอื้อมีมารดาชื่อว่าอะไร?,กันเกิดเมื่อวันที่ 23 ตุลาคม พ.ศ. 2533 จังหวัด...,"{'text': ['นางวรรณา อินทร์ใจเอื้อ', 'นางวรรณา ...",tydiqa,5e2f4e69eb82918a55969ec99b1c93ae469c264c81c77a...


In [3]:
def augment(dataset, aug_fnc, col_name):
    data = []
    for _, row in tqdm(dataset.iterrows(), total=len(dataset)):
        augmented = aug_fnc(row['question'])
        for sent in augmented:
            sent = ''.join(sent)
            data.append({
                'id': row['id'],
                col_name: sent,
            })
        del augmented

    gc.collect()
    return pd.DataFrame(data)

## WordNetAug

In [4]:
wordnet = WordNetAug()

In [6]:
wordnet_aug = augment(dataset, wordnet.augment, 'th_wordnet_aug')
wordnet_aug

  0%|          | 0/16980 [00:00<?, ?it/s]

In [ ]:
wordnet_aug.to_csv('data/02_th_wordnet_aug.csv', index=False)

## Thai2fitAug

In [6]:
thai2fit = word2vec.Thai2fitAug()

Corpus: thai2fit_wv
- Downloading: thai2fit_wv 0.1


  0%|          | 0/62452646 [00:00<?, ?it/s]

In [ ]:
thai2fit_aug = augment(dataset, lambda x: thai2fit.augment(x, n_sent=6, p=0.7), 'th_thai2fit_aug')

In [ ]:
thai2fit_aug.to_csv('data/02_th_thai2fit_aug.csv', index=False)

# LTW2VAug

In [18]:
ltw2v = word2vec.LTW2VAug()

Corpus: ltw2v
- Downloading: ltw2v 0.1


  0%|          | 0/1178368364 [00:00<?, ?it/s]

In [ ]:
ltw2v_aug = augment(dataset, lambda x: ltw2v.augment(x, n_sent=6, p=0.7), 'th_ltw2v_aug')

In [ ]:
ltw2v_aug.to_csv('data/02_th_ltw2v_aug.csv', index=False)

## FastTextAug

Download model from https://fasttext.cc/docs/en/crawl-vectors.html

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.th.300.bin.gz

In [ ]:
fasttext = lm.FastTextAug('cc.th.300.bin.gz')

In [ ]:
# test out first

In [ ]:
fasttext_aug = augment(dataset, lambda x: fasttext.augment(x, n_sent=6, p=0.7), 'th_fasttext_aug')

## Thai2Transformers

In [ ]:
thai2transformers = lm.Thai2transformersAug()

In [ ]:
thai2transformers_aug = augment(dataset, lambda x: thai2transformers.augment(x, num_replace_tokens=6), 'th_thai2transformers_aug')

In [ ]:
thai2transformers_aug.to_csv('data/02_th_thai2transformers_aug.csv', index=False)

# BPEmbAug

In [28]:
from pythainlp.augment.word2vec import bpemb_wv 

In [30]:
bpemb = bpemb_wv.BPEmbAug()

downloading https://nlp.h-its.org/bpemb/th/th.wiki.bpe.vs100000.model


100%|██████████| 3210051/3210051 [00:01<00:00, 2566332.97B/s]


downloading https://nlp.h-its.org/bpemb/th/th.wiki.bpe.vs100000.d300.w2v.bin.tar.gz


100%|██████████| 112536302/112536302 [00:32<00:00, 3494093.78B/s]


In [ ]:
bpemb_aug = augment(dataset, lambda x: bpemb.augment(x, n_sent=6, p=0.5), 'th_bpemb_aug')

In [ ]:
bpemb_aug.to_csv('data/02_th_bpemb_aug.csv', index=False)